Hoon Unit Test Framework
---

Provisions:
- header including imports and definitions
- body including field of tests in terms of header values

In [1]:
header_global = \
'''
/+  *test
/+  *lagoon
^|
|_  $:  atol=_.1e-3          :: absolute tolerance for precision of operations
        rtol=_.1e-5          :: relative tolerance for precision of operations
    ==
::  Auxiliary tools
++  is-equal
  |=  [a=ray:la b=ray:la]  ^-  tang
  ?:  =(a b)  ~
  :~  [%palm [": " ~ ~ ~] [leaf+"expected" "{<a>}"]]
      [%palm [": " ~ ~ ~] [leaf+"actual  " "{<b>}"]]
  ==
::
++  is-close
  |=  [a=ray:la b=ray:la =term]  ^-  tang
  ?:  (all:la (is-close:la a b term [atol rtol]))  ~
  :~  [%palm [": " ~ ~ ~] [leaf+"expected" "{<a>}"]]
      [%palm [": " ~ ~ ~] [leaf+"actual  " "{<b>}"]]
  ==
'''

In [2]:
header_vars = \
'''
=/  meta-1x1-5=meta:la  [~[1 1] 5 %uint ~]
=/  meta-2x2-4=meta:la  [~[2 2] 4 %uint ~]
=/  meta-nxn-3=meta:la  [~[16 16] 3 %uint ~]
=/  assay-1x1-5=ray:la  (spac:la [meta=meta-1x1-5 data=0x1])
=/  assay-2x2-4=ray:la  (spac:la [meta=meta-2x2-4 data=0x1.0002.0003.0004])
=/  assay-nxn-3=ray:la  (spac:la [meta=meta-nxn-3 data=0xfffe.fdfc.fbfa.f9f8.f7f6.f5f4.f3f2.f1f0.efee.edec.ebea.e9e8.e7e6.e5e4.e3e2.e1e0.dfde.dddc.dbda.d9d8.d7d6.d5d4.d3d2.d1d0.cfce.cdcc.cbca.c9c8.c7c6.c5c4.c3c2.c1c0.bfbe.bdbc.bbba.b9b8.b7b6.b5b4.b3b2.b1b0.afae.adac.abaa.a9a8.a7a6.a5a4.a3a2.a1a0.9f9e.9d9c.9b9a.9998.9796.9594.9392.9190.8f8e.8d8c.8b8a.8988.8786.8584.8382.8180.7f7e.7d7c.7b7a.7978.7776.7574.7372.7170.6f6e.6d6c.6b6a.6968.6766.6564.6362.6160.5f5e.5d5c.5b5a.5958.5756.5554.5352.5150.4f4e.4d4c.4b4a.4948.4746.4544.4342.4140.3f3e.3d3c.3b3a.3938.3736.3534.3332.3130.2f2e.2d2c.2b2a.2928.2726.2524.2322.2120.1f1e.1d1c.1b1a.1918.1716.1514.1312.1110.0f0e.0d0c.0b0a.0908.0706.0504.0302.0100])
=/  canon-1x1-5=ray:la  [meta=meta-1x1-5 data=0x1.0000.0001]
=/  canon-2x2-4=ray:la  [meta=meta-2x2-4 data=0x1.0001.0002.0003.0004]
=/  canon-nxn-3=ray:la  [meta=meta-nxn-3 data=0x1.fffe.fdfc.fbfa.f9f8.f7f6.f5f4.f3f2.f1f0.efee.edec.ebea.e9e8.e7e6.e5e4.e3e2.e1e0.dfde.dddc.dbda.d9d8.d7d6.d5d4.d3d2.d1d0.cfce.cdcc.cbca.c9c8.c7c6.c5c4.c3c2.c1c0.bfbe.bdbc.bbba.b9b8.b7b6.b5b4.b3b2.b1b0.afae.adac.abaa.a9a8.a7a6.a5a4.a3a2.a1a0.9f9e.9d9c.9b9a.9998.9796.9594.9392.9190.8f8e.8d8c.8b8a.8988.8786.8584.8382.8180.7f7e.7d7c.7b7a.7978.7776.7574.7372.7170.6f6e.6d6c.6b6a.6968.6766.6564.6362.6160.5f5e.5d5c.5b5a.5958.5756.5554.5352.5150.4f4e.4d4c.4b4a.4948.4746.4544.4342.4140.3f3e.3d3c.3b3a.3938.3736.3534.3332.3130.2f2e.2d2c.2b2a.2928.2726.2524.2322.2120.1f1e.1d1c.1b1a.1918.1716.1514.1312.1110.0f0e.0d0c.0b0a.0908.0706.0504.0302.0100]
'''

We need to be able to provide the answer in a NumPy format, then get it to a Hoon format when we build.


Test fields are named as:

- `assay` for argument inputs
- `canon` for expected values
- `label` as an identifier
- `dims` in `mxn` format
- `bitwidth` in $2^{\texttt{bitwidth}}$ format
- `suffix` as one of `r, u, i, c, p, f`

Some representative variable names include:

```
assay-ones-2x2-3u   for ~[~[1 1] ~[1 1]] in 8-bit uint
assay-zeros-3x3-5s  for ~[~[0 0 0] ~[0 0 0] ~[0 0 0]] in 32-bit 2s-complement
canon-ones-1x1-6r   for ~[~[1]] in 64-bit double-precision floating-point (real)
```

In [3]:
import numpy as np
from itertools import product

```
  +$  ray               ::  $ray:  n-dimensional array
    $:  =meta           ::  descriptor
        data=@ux        ::  data, row-major order
    ==
  ::
  +$  meta              ::  $meta:  metadata for a $ray
    $:  shape=(list @)  ::  list of dimension lengths
        =bloq           ::  logarithm of bitwidth
        =kind           ::  name of data type
        prec=(unit @)   ::  fixed-precision scale
    ==
```

In [189]:
def float2hoon(number, bloq):
    if bloq == 4:
        return f'.~~{number}'
    if bloq == 5:
        return f'.{number}'
    if bloq == 6:
        return f'.~{number}'
    if bloq == 7:
        return f'.~~~{number}'
    raise Exception(f'invalid bloq {bloq} for {number}')

In [96]:
def dtype2kind(dtype):
    int_types = [np.uint8, np.uint16, np.uint32, np.uint64]
    return '%uint' if dtype in int_types else '%real'

def np2baum(array):
    shape = '~[' + ' '.join([str(i) for i in array.shape]) + ']'
    bloq = int(np.log2(array.dtype.itemsize * 8))
    kind = dtype2kind(array.dtype)
    
    if kind == '%uint':
        baum = str(array).replace('\n','').replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('[','~[').replace('.', '')
    else:
        baum = '~['
        # 1D case
        if array.ndim == 1:
            for el in array:
                baum += float2hoon(el, bloq) + ' '
            baum = baum.strip() + ']'
            return f'[meta=[shape={shape} bloq={bloq} kind={kind} prec=~] baum={baum}]'        

        # 2D case
        for row in array:
            if baum[-1] == ']':  baum += ' '
            baum += '~['
            for el in row:
                baum += float2hoon(el, bloq) + ' '
            baum = baum.strip() + ']'
        baum = baum.strip() + ']'
        
    return f'[meta=[shape={shape} bloq={bloq} kind={kind} prec=~] baum={baum}]'

In [97]:
i = list(range(1,4))
j = i.copy()
bitwidth = list(range(3,8))
suffix = list('ru')

def bits2dtype(bitwidth, suffix):
    if bitwidth == 3 and suffix == 'u':
        return np.uint8
    if bitwidth == 4 and suffix == 'u':
        return np.uint16
    if bitwidth == 5 and suffix == 'u':
        return np.uint32
    if bitwidth == 6 and suffix == 'u':
        return np.uint64
    if bitwidth == 4 and suffix == 'r':
        return np.float16
    if bitwidth == 5 and suffix == 'r':
        return np.float32
    if bitwidth == 6 and suffix == 'r':
        return np.float64
# NumPy float128 actually long double = 80 bits
#     if bitwidth == 7 and suffix = 'r':
#         return np.float128
    raise Exception('invalid type combination')

inputs = {}
for i,j,bitwidth,suffix in product(i,j,bitwidth,suffix):
    try:
        varname = f'input-ones-{i}x{j}-{bitwidth}{suffix}'
        inputs[varname] = np.ones((i,j), dtype=bits2dtype(bitwidth,suffix))
    except Exception:
        continue

In [98]:
i = list(range(1,4))
j = i.copy()
bitwidth = list(range(3,8))
suffix = list('ru')

for i,j,bitwidth,suffix in product(i,j,bitwidth,suffix):
    try:
        varname = f'input-twos-{i}x{j}-{bitwidth}{suffix}'
        inputs[varname] = 2*np.ones((i,j), dtype=bits2dtype(bitwidth,suffix))
    except Exception:
        continue

In [99]:
np.ones((1,)).ndim

1

In [100]:
kinds = {
    'r': '%real',
    'u': '%uint',
    's': '%int2',
    'c': '%cplx',
    'p': '%unum',
    'f': '%fixp',
}

def label2meta(label):
    shape_offset = label.find('-', 6)
    shape_end = label.find('-', shape_offset+1)
    shape = ([int(x) for x in label[shape_offset+1:shape_end].split('x')])
    
    bloq = int(label[shape_end+1:-1])
    
    kind = kinds[label[-1]]
    
    return (shape, bloq, label[-1], 0)

In [133]:
# single-argument tests

def f2u(eff):
    if eff == np.float16:
        return np.uint16
    if eff == np.float32:
        return np.uint32
    if eff == np.float64:
        return np.uint64
    return eff

'''
-1 = single tuple (like np.array)
1 = one arg (like iota)
'''

hoon2np = {
    'eye': (1, np.eye),
    'zeros': (-1, np.zeros),
    'ones': (-1, np.ones),
    #'iota': (1, lambda x,dtype: np.array(range(1, x+1),dtype=f2u(dtype))-1),  # 1D only
    # TODO linspace etc.
}

canons = {}

from textwrap import dedent

for ilabel, fn in product(inputs, hoon2np):
    meta = label2meta(ilabel)
    offset = ilabel.find('-', 6)
    clabel = 'canon-' + fn + ilabel[offset:]
    if clabel in canons:  continue
    
    # single tuple inputs
    if hoon2np[fn][0] == -1:
        canons[clabel] = hoon2np[fn][1](meta[0], dtype=bits2dtype(meta[1], meta[2]))
    
    # single scalar input
    elif hoon2np[fn][0] == 1 and (meta[0][0] == meta[0][1]):
        canons[clabel] = hoon2np[fn][1](meta[0][0], dtype=bits2dtype(meta[1], meta[2]))

    # other inputs (like two args)
    elif fn[0] == len(meta[0]):
        canons[clabel] = hoon2np[fn][1](meta[0], dtype=bits2dtype(meta[1], meta[2]))
    
    if clabel in canons:
        alabel = clabel.replace('canon', 'assay')
        print(dedent(f'''
        ++  test-{clabel[6:]}  ^-  tang
          =/  {ilabel}  (en-ray:la {np2baum(inputs[ilabel])})
          =/  {clabel}  (en-ray:la {np2baum(canons[clabel])})
          =/  {alabel}  ({fn}:la meta.{ilabel})
          %+  is-equal
            {clabel}
          {alabel}'''))


++  test-eye-1x1-3u  ^-  tang
  =/  input-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  canon-eye-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  assay-eye-1x1-3u  (eye:la meta.input-ones-1x1-3u)
  %+  is-equal
    canon-eye-1x1-3u
  assay-eye-1x1-3u

++  test-zeros-1x1-3u  ^-  tang
  =/  input-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  canon-zeros-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[0]]])
  =/  assay-zeros-1x1-3u  (zeros:la meta.input-ones-1x1-3u)
  %+  is-equal
    canon-zeros-1x1-3u
  assay-zeros-1x1-3u

++  test-ones-1x1-3u  ^-  tang
  =/  input-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  canon-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  assay-ones-1x1-3u  (ones:la meta.input-ones-1x1-3u)
  %+  is-equal
    canon-ones-1x1-

In [195]:
# two same-sized arguments tests

def f2u(eff):
    if eff == np.float16:
        return np.uint16
    if eff == np.float32:
        return np.uint32
    if eff == np.float64:
        return np.uint64
    return eff

def u2f(you):
    if you == np.uint16:
        return np.float16
    if you == np.uint32:
        return np.float32
    if you == np.uint64:
        return np.floatt64
    return you

'''
-1 = single tuple (like np.array)
1 = one arg (like iota)
'''

hoon2np = {
    'add': (2, np.add),
    'sub': (2, np.subtract),
    'mul': (2, np.multiply),
    'div': (2, np.divide),
    'mod': (2, np.mod),
    #  flip for loobean logic
    'gth': (2, np.less_equal),
    'gte': (2, np.less),
    'lth': (2, np.greater_equal),
    'lte': (2, np.greater),
}

def one_float(dtype):
    if dtype == np.float16:
        return 0x3c00
    elif dtype == np.float32:
        return 0x3f800000
    elif dtype == np.float64:
        return 0x3ff0000000000000
    else:
        return 0x1

canons = {}

from textwrap import dedent

for fn, ilabel, jlabel in product(hoon2np, inputs, inputs):
    imeta = label2meta(ilabel)
    jmeta = label2meta(jlabel)
    if imeta != jmeta:  continue
    offset = ilabel.find('-', 6)
    clabel = 'canon-' + fn + ilabel[offset:]
    if clabel in canons:  continue
    if fn in ['mod'] and inputs[ilabel].dtype in [np.float16, np.float32, np.float64]: continue
    
    # other inputs (like two args)
    if fn in ['gth', 'gte', 'lth', 'lte'] and inputs[ilabel].dtype in [np.float16, np.float32, np.float64]:
        canons[clabel] = hoon2np[fn][1](inputs[ilabel], inputs[jlabel]).astype(inputs[ilabel].dtype)
        original = canons[clabel].dtype
        canons[clabel].dtype = f2u(original)
        canons[clabel][canons[clabel] == one_float(original)] = 0x1
        canons[clabel].dtype = original
    else:
#    elif fn in ['div']:
        canons[clabel] = hoon2np[fn][1](inputs[ilabel], inputs[jlabel]).astype(inputs[ilabel].dtype)
#     else:
#         canons[clabel] = hoon2np[fn][1](inputs[ilabel], inputs[jlabel])
    
    if clabel in canons:
        alabel = clabel.replace('canon', 'assay')
#         if fn in ['gth', 'gte', 'lth', 'lte']:
#             # f'[meta=[shape={shape} bloq={bloq} kind={kind} prec=~] baum={baum}]'
#             offset = clabel_.find('kind=') + 5
#             clabel_ = clabel_[:offset] + '%uint' + clabel_[offset+5:]
#         print(clabel, canons[clabel], canons[clabel].dtype)
        print(dedent(f'''
        ++  test-{clabel[6:]}  ^-  tang
          =/  {ilabel}  (en-ray:la {np2baum(inputs[ilabel])})
          =/  {"j"+jlabel[1:]}  (en-ray:la {np2baum(inputs[jlabel])})
          =/  {clabel}  (en-ray:la {np2baum(canons[clabel])})
          =/  {alabel}  ({fn}:la {ilabel} {"j"+jlabel[1:]})
          %+  is-equal
            {clabel}
          {alabel}''').replace('e-0','e-'))


++  test-add-1x1-3u  ^-  tang
  =/  input-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  jnput-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  canon-add-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[2]]])
  =/  assay-add-1x1-3u  (add:la input-ones-1x1-3u jnput-ones-1x1-3u)
  %+  is-equal
    canon-add-1x1-3u
  assay-add-1x1-3u

++  test-add-1x1-4r  ^-  tang
  =/  input-ones-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~1.0]]])
  =/  jnput-ones-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~1.0]]])
  =/  canon-add-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~2.0]]])
  =/  assay-add-1x1-4r  (add:la input-ones-1x1-4r jnput-ones-1x1-4r)
  %+  is-equal
    canon-add-1x1-4r
  assay-add-1x1-4r

++  test-add-1x1-4u  ^-  tang
  =/  input-ones-1x1-4u  (en-ray:la [meta=[shape=~[1 1] bloq=4 k


++  test-mul-1x1-3u  ^-  tang
  =/  input-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  jnput-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  canon-mul-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  assay-mul-1x1-3u  (mul:la input-ones-1x1-3u jnput-ones-1x1-3u)
  %+  is-equal
    canon-mul-1x1-3u
  assay-mul-1x1-3u

++  test-mul-1x1-4r  ^-  tang
  =/  input-ones-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~1.0]]])
  =/  jnput-ones-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~1.0]]])
  =/  canon-mul-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~1.0]]])
  =/  assay-mul-1x1-4r  (mul:la input-ones-1x1-4r jnput-ones-1x1-4r)
  %+  is-equal
    canon-mul-1x1-4r
  assay-mul-1x1-4r

++  test-mul-1x1-4u  ^-  tang
  =/  input-ones-1x1-4u  (en-ray:la [meta=[shape=~[1 1] bloq=4 k


++  test-mod-1x1-3u  ^-  tang
  =/  input-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  jnput-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  canon-mod-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[0]]])
  =/  assay-mod-1x1-3u  (mod:la input-ones-1x1-3u jnput-ones-1x1-3u)
  %+  is-equal
    canon-mod-1x1-3u
  assay-mod-1x1-3u

++  test-mod-1x1-4u  ^-  tang
  =/  input-ones-1x1-4u  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%uint prec=~] baum=~[~[1]]])
  =/  jnput-ones-1x1-4u  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%uint prec=~] baum=~[~[1]]])
  =/  canon-mod-1x1-4u  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%uint prec=~] baum=~[~[0]]])
  =/  assay-mod-1x1-4u  (mod:la input-ones-1x1-4u jnput-ones-1x1-4u)
  %+  is-equal
    canon-mod-1x1-4u
  assay-mod-1x1-4u

++  test-mod-1x1-5u  ^-  tang
  =/  input-ones-1x1-5u  (en-ray:la [meta=[shape=~[1 1] bloq=5 kind=%uint prec=


++  test-gte-1x1-3u  ^-  tang
  =/  input-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  jnput-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  canon-gte-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[0]]])
  =/  assay-gte-1x1-3u  (gte:la input-ones-1x1-3u jnput-ones-1x1-3u)
  %+  is-equal
    canon-gte-1x1-3u
  assay-gte-1x1-3u

++  test-gte-1x1-4r  ^-  tang
  =/  input-ones-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~1.0]]])
  =/  jnput-ones-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~1.0]]])
  =/  canon-gte-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~0.0]]])
  =/  assay-gte-1x1-4r  (gte:la input-ones-1x1-4r jnput-ones-1x1-4r)
  %+  is-equal
    canon-gte-1x1-4r
  assay-gte-1x1-4r

++  test-gte-1x1-4u  ^-  tang
  =/  input-ones-1x1-4u  (en-ray:la [meta=[shape=~[1 1] bloq=4 k


++  test-lte-1x1-3u  ^-  tang
  =/  input-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  jnput-ones-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[1]]])
  =/  canon-lte-1x1-3u  (en-ray:la [meta=[shape=~[1 1] bloq=3 kind=%uint prec=~] baum=~[~[0]]])
  =/  assay-lte-1x1-3u  (lte:la input-ones-1x1-3u jnput-ones-1x1-3u)
  %+  is-equal
    canon-lte-1x1-3u
  assay-lte-1x1-3u

++  test-lte-1x1-4r  ^-  tang
  =/  input-ones-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~1.0]]])
  =/  jnput-ones-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~1.0]]])
  =/  canon-lte-1x1-4r  (en-ray:la [meta=[shape=~[1 1] bloq=4 kind=%real prec=~] baum=~[~[.~~0.0]]])
  =/  assay-lte-1x1-4r  (lte:la input-ones-1x1-4r jnput-ones-1x1-4r)
  %+  is-equal
    canon-lte-1x1-4r
  assay-lte-1x1-4r

++  test-lte-1x1-4u  ^-  tang
  =/  input-ones-1x1-4u  (en-ray:la [meta=[shape=~[1 1] bloq=4 k

In [188]:
val = np.ones((1,1), dtype=np.float16)
val.dtype = np.uint16
print(val)
val[val == 15360] = 1
val.dtype = np.uint16
print(val)

[[15360]]
[[1]]


In [168]:
val = np.ones((2,2), dtype=np.float32)
val.dtype = np.uint8
val[val == 1] = 0x1
val

array([[  0,   0, 128,  63,   0,   0, 128,  63],
       [  0,   0, 128,  63,   0,   0, 128,  63]], dtype=uint8)

In [113]:
i = list(range(1,4))
j = i.copy()
bitwidth = list(range(3,8))
suffix = list('ru')

canons = {}
for i,j,bitwidth,suffix,label in product(i,j,bitwidth,suffix,hoon2np.keys()):
    try:
        varname = f'canon-{label}-{i}x{j}-{bitwidth}{suffix}'
        canons[varname] = hoon2np[label][1]((i,j))
        canons[varname].dtype = bits2dtype(bitwidth,suffix)
    except Exception:
        continue

In [114]:
o

NameError: name 'o' is not defined

In [115]:
np.eye((2,3))

TypeError: 'tuple' object cannot be interpreted as an integer

In [ ]:
# 4D cases

kind = ['assay', 'canon']
label = []
i = list(range(1,4))
j = i.copy()
k = i.copy()
l = i.copy()
bitwidth = list(range(3,8))
suffix = list('ruicpf')

for kind,i,j,k,l,bitwidth,suffix in product(kind,i,j,k,l,bitwidth,suffix):
    print(f'{kind}-{i}x{j}x{k}x{l}-{bitwidth}{suffix}')